### Basit e-mail gönderimi

#### gmail ile deneme:

In [2]:
import ssl # ssl: simple socket layer
import smtplib # smtp: simple mail transfer protocol

In [4]:
#kullanici = " "
#sifre = " "

In [6]:
alici=kullanici

mesaj = "Deneme mesajı."

context=ssl.create_default_context() #ezber

In [7]:
#gmail smpt adresi https://support.google.com/mail/answer/7126229?hl=tr

port = 465
host = "smtp.gmail.com"

eposta_sunucu = smtplib.SMTP_SSL(host = host, port = port, context = context) 
eposta_sunucu.login(kullanici, sifre)
eposta_sunucu.sendmail(kullanici, alici, mesaj)


SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials ag14-20020a1709069a8e00b00a3f963c66a8sm89568ejc.121 - gsmtp')

In [10]:
# 2023 itibariyle güvenlik sebepleri sonucunda google buna izin vermiyor

#### Outlook ile deneme:

In [ ]:
# outlook smtp adresi 
# https://support.microsoft.com/tr-tr/office/outlook-com-için-pop-imap-ve-smtp-ayarları-d088b986-291d-42b8-9564-9c414e2aa040

#IMAP sunucusu adı (host)   outlook.office365.com
#IMAP bağlantı noktası (port)   993

#SMTP sunucusu adı (host)   smtp-mail.outlook.com
#SMTP bağlantı noktası (port)   587

In [57]:
import ssl 
import smtplib 

In [4]:
import smtplib
import ssl
import getpass
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

kullanici = input("E-posta adresinizi giriniz: ")
sifre = getpass.getpass("Şifrenizi giriniz: ")

context = ssl.create_default_context()

port = 587
host = "smtp-mail.outlook.com"
eposta_sunucu = smtplib.SMTP(host=host, port=port)
eposta_sunucu.starttls(context=context)
eposta_sunucu.login(kullanici, sifre)

alici = input("Alıcı e-posta adresini giriniz: ")
subject = input("Konu: ")
icerik = input("Mesaj: ")

html_icerik = """\
<html>
  <body>
    <p>{}</p>
  </body>
</html>
""".format(icerik)
mesaj = MIMEMultipart()
mesaj.attach(MIMEText(html_icerik, 'html'))
mesaj["Subject"] = subject

eposta_sunucu.sendmail(kullanici, alici, mesaj.as_string())
eposta_sunucu.quit()

print("Mail gönderme başarılı.")

E-posta adresinizi giriniz:  denemeysb1@outlook.com
Şifrenizi giriniz:  ········
Alıcı e-posta adresini giriniz:  denemeysb1@outlook.com
Konu:  Python'dan mail gönderiyorum!
Mesaj:  Bu maili kendime pythondan yazdığım kod aracılığıyla gönderiyorum.


Mail gönderme başarılı.


### Eklentili e-mail gönderimi

In [55]:
from email.mime.base import MIMEBase
from email import encoders #eklentiler de string şekliyle gönderilecek ancak stringe encode edilmeli 
                           #mesela jpg stringe dönüşecek öyle gidecek

### önceki kısımla aynı
import smtplib
import ssl
import getpass
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

kullanici = input("E-posta adresinizi giriniz: ")

while not ("@" and ".") in kullanici:
    print("E-posta adresiniz hatalı veya eksik girildi.")
    kullanici = input("E-posta adresinizi kontrol ederek tekrar deneyiniz: ")
else:
    pass
  
sifre = getpass.getpass("Şifrenizi giriniz: ")

context = ssl.create_default_context()

port = 587
host = "smtp-mail.outlook.com"
eposta_sunucu = smtplib.SMTP(host=host, port=port)
eposta_sunucu.starttls(context=context)
while True:
    try:
        eposta_sunucu.login(kullanici, sifre)
        print("Giriş başarılı.")
        break
    except:
        print("Giriş başarısız. \nE-posta adresinizi veya giriş şifrenizi kontrol ederek tekrar deneyiniz.")
        kullanici = input("E-posta adresinizi giriniz: ")
        sifre = getpass.getpass("Şifrenizi giriniz: ")
        
alici = input("Alıcı e-posta adresini giriniz: ")

while not ("@" and ".") in alici:
    print("Alıcı e-posta adresi hatalı veya eksik girildi.")
    alici = input("Alici e-posta adresini kontrol ederek tekrar deneyiniz: ")
else:
    pass
    
subject = input("Konu: ")
icerik = input("Mesaj: ")

html_icerik = """\
<html>
  <body>
    <p>{}</p>
  </body>
</html>
""".format(icerik)
#bu kısmı ekliyoruz
posta = MIMEMultipart()
posta["Subject"] = subject
posta["From"]=kullanici
posta["To"]=alici # bu kısımda çoklu alıcı varsa [kullanici, kullanici2, oguzyilmaz@gmail.com] vs. diye liste halinde ekleyip uzatabiliriz
posta.attach(MIMEText(html_icerik, 'html'))

eklenti = input("Eklenti adresini giriniz:")

mimetype, _ = mimetypes.guess_type(eklenti) #eklenti uzantısı olmadan açılamaz. MIME tipini belirliyoruz
if mimetype is None:
    mimetype = "application/octet-stream"

with(open(eklenti,"rb")) as eklenti_dosyasi: #read okumasını istiyoruz
    #payload = MIMEBase("application", "octate-stream")
    #üstteki susturdum çünkü mimetype = "application/octet-stream" dedik
    payload = MIMEBase(*mimetype.split("/")) 
    payload.set_payload((eklenti_dosyasi).read())
    encoders.encode_base64(payload)

    dosya_adı = eklenti.split("/")[-1] #dosya adından uzantısını alıp ekliyoruz
    payload.add_header("Content-Decomposition","attachment", filename=dosya_adı)
    posta.attach(payload)

    posta_str=posta.as_string() #bu kısmı aşağıda parantez içinde de yapabiliriz
                                #eposta_sunucu.sendmail(kullanici, alici, mesaj.as_string()) şeklinde

eposta_sunucu.sendmail(kullanici, alici, posta_str)
eposta_sunucu.quit()

print("Mail gönderme başarılı.")

E-posta adresinizi giriniz:  denemeysb1@outlook


E-posta adresiniz hatalı veya eksik girildi.


E-posta adresinizi kontrol ederek tekrar deneyiniz:  denemeysb1@outlook.com
Şifrenizi giriniz:  ········


Giriş başarısız. 
E-posta adresinizi veya giriş şifrenizi kontrol ederek tekrar deneyiniz.


E-posta adresinizi giriniz:  denemeysb1@outlook.com
Şifrenizi giriniz:  ········


Giriş başarısız. 
E-posta adresinizi veya giriş şifrenizi kontrol ederek tekrar deneyiniz.


E-posta adresinizi giriniz:  denemeysb1@outlook.com
Şifrenizi giriniz:  ········


Giriş başarılı.


Alıcı e-posta adresini giriniz:  denemeysb1@outlook.com
Konu:  Python'dan eklentili mail gönderiyorum!
Mesaj:  Bu eklentili maili kendime pythondan yazdığım kod aracılığıyla gönderiyorum.
Eklenti adresini giriniz: /Users/yavuzsebe/Library/CloudStorage/OneDrive-YildizTechnicalUniversity/MacbookAirScreenshots/gui.png


Mail gönderme başarılı.


### E-mail okuma

In [1]:
pip install imap_tools
#github.com/ikvk/imap_tools

Note: you may need to restart the kernel to use updated packages.


In [2]:
# outlook smtp adresi 
# https://support.microsoft.com/tr-tr/office/outlook-com-için-pop-imap-ve-smtp-ayarları-d088b986-291d-42b8-9564-9c414e2aa040

#IMAP sunucusu adı (host)   outlook.office365.com
#IMAP bağlantı noktası (port)   993

#SMTP sunucusu adı (host)   smtp-mail.outlook.com
#SMTP bağlantı noktası (port)   587

In [94]:
kullanici = " "
sifre = " "

In [88]:
import imaplib
import email
import datetime
from email import policy
from email.parser import BytesParser
from imap_tools import AND # AND, OR, Q, NOT gibi çokça şey eklenebilir
from bs4 import BeautifulSoup  # BeautifulSoup HTML parsing için kullanılacak

# IMAP4_SSL kullanarak bağlantı kurulacak
mail = imaplib.IMAP4_SSL("outlook.office365.com")
mail.login(kullanici, sifre)
"""
with imaplib.IMAP4_SSL("outlook.office365.com").login(kullanici, sifre) as mail:
    -kodun kalanı- #indent halinde
    #bu şekilde daha kısa şekilde yazılabilir
"""
# gelen kutusundaki mailleri almak için
mail.select("inbox")

# belirli bir tarihten itibaren gelen mailleri almak için

kriter = AND(date_gte=datetime.date(2024, 2, 24), from_=kullanici)

# kriter i bytes türüne çevirme
kriter = bytes(str(kriter), 'utf-8')

#kriter = f'(SINCE {tarih.strftime("%d-%b-%Y")})' # f'(xxx{y}xxx{z}xxx)' aynı format gibi çalışır (xxx{}xx{}x).format(y,z)
# kriter=AND(date_gte=datetime.date(2024,2,24), from_=kullanici) böyle de yazım mevcut
#gte: greater than equal belli bir tarih ve sonrasını getirmesini sağlayabiliriz datetime modülü gerekir
#kullanici kişisinden mailleri al

# IMAP4.uid fonksiyonunu çağırma
result, data = mail.uid('search', None, kriter)

#bu kısım metnin html den decode edilmesi ve yazdırılması üzerine
if result == 'OK':
    for num in data[0].split():
        result, msg_data = mail.uid('fetch', num, '(RFC822)')
        if result == 'OK':
            raw_email = msg_data[0][1]
            parsed_email = BytesParser(policy=policy.default).parsebytes(raw_email)
    
            print("Konu:", parsed_email["subject"])
            print("Gönderen:", parsed_email["from"])
            print("Tarih:", parsed_email["date"])
    
            #metin içeriğini al
            metin = None
            for part in parsed_email.iter_parts():
                if part.get_content_type() == "text/plain":
                    metin = part.get_payload(decode=True)
                    break
                elif part.get_content_type() == "text/html":
                    metin = part.get_payload(decode=True)
                    break
    
            #eğer metin içeriği varsa, HTML etiketlerini temizler
            if metin:
                charset = parsed_email.get_charset() or "utf-8"
                try:
                    metin = metin.decode(charset, "ignore")
                    # HTML etiketlerini temizleme kısmı
                    soup = BeautifulSoup(metin, 'html.parser')
                    metin = soup.get_text(separator='\n', strip=True)
                    print("Metin:", metin)
                except UnicodeDecodeError:
                    print("Metin çözümlenemedi.")
    
            print("------------------------------------------")
            
#mail.logout()

Konu: Python'dan mail gönderiyorum!
Gönderen: denemeysb1@outlook.com
Tarih: Sat, 24 Feb 2024 11:35:29 +0000
Metin: Bu maili kendime pythondan yazdığım kod aracılığıyla gönderiyorum.
------------------------------------------
Konu: Python'dan eklentili mail gönderiyorum!
Gönderen: denemeysb1@outlook.com
Tarih: Sun, 25 Feb 2024 21:20:10 +0000
Metin: Bu eklentili maili kendime pythondan yazdığım kod aracılığıyla gönderiyorum.
------------------------------------------


In [89]:
import datetime
from imap_tools import MailBox
from imap_tools import AND # and,or,q gibi çokça şey eklenebilir

with MailBox("outlook.office365.com").login(kullanici, sifre, initial_folder="INBOX") as posta_kutusu:
    kriter=AND(date_gte=datetime.date(2024,2,24), from_=kullanici) 
    
    for msg in posta_kutusu.fetch(kriter):
        print(msg.text)

In [92]:
def dosya_isminden_mail_bul(eposta_kutusu_param, dosya_ismi_param, kriter_param):
    kriter_param = 'HEADER FROM {} SINCE 24-Feb-2024'.format(kullanici)
    for mesaj in eposta_kutusu_param.fetch(kriter_param, 'BODY.PEEK[]'):
        if mesaj.attachments:
            for ek in mesaj.attachments:
                if dosya_ismi_param == ek.filename:
                    return f"{dosya_ismi_param} isimli dosya, {mesaj.date_str} tarihli ve {mesaj.subject} başlıklı e-postadadır."
    return f'{dosya_ismi_param} isimli dosya bulunamadı.'


In [ ]:
kriter = AND(date_gte=datetime.date(2024, 2, 24), from_=kullanici)
dosya_ismi = "gui.png"
eposta_kutusu = mail
dosya_isminden_mail_bul(eposta_kutusu, dosya_ismi, kriter)